In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetLarge,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexPE"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (331,331,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(331, 331))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 331, 331, 3)

In [5]:
# Creating base_model
image_shape = (331, 331, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetLarge(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 4032)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 4032)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min


[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.834638, total= 1.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.803922, total= 1.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.822266, total= 1.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.811024, total= 1.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.826561, total= 1.7min
[CV] ........................... C=0.01, score=0.833828, total= 1.7min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.826389, total= 1.7min
[CV] C=10.0 ..........................................................
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  8.1min


[CV] ............................ C=0.1, score=0.867126, total= 7.9min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.864706, total= 8.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.903131, total= 8.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.890748, total= 8.2min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.885686, total= 8.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.891972, total= 8.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.887450, total= 8.6min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.912109, total= 9.6min
[CV] ................... C=1000000000.0, score=0.908824, total= 8.5min
[CV] ................... C=1000000000.0, score=0.890748, total= 8.4min
[CV] ................... C=1000000000.0, score=0.917323, total= 8.6min
[CV] ................... C=1000000000.0, score=0.900099, total= 8.5min
[CV] ................... C=1000000000.0, score=0.917740, total= 8.2min
[CV] ................... C=1000000000.0, score=0.919643, total= 8.0min
[CV] ................... C=1000000000.0, score=0.898608, total= 7.5min
[CV] ................... C=1000000000.0, score=0.911355, total= 7.2min
[CV] .................. C=10000000000.0, score=0.911133, total= 6.7min
[CV] .................. C=10000000000.0, score=0.905882, total= 6.3min
[CV] .................. C=10000000000.0, score=0.918787, total= 6.6min
[CV] .................. C=10000000000.0, score=0.905045, total= 6.3min
[CV] .................. C=10000000000.0, score=0.922244, total= 6.4min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 62.3min remaining:  2.5min


[CV] .................. C=10000000000.0, score=0.889764, total= 6.4min
[CV] .................. C=10000000000.0, score=0.916667, total= 6.0min
[CV] .................. C=10000000000.0, score=0.922696, total= 6.1min
[CV] .................. C=10000000000.0, score=0.907371, total= 5.7min
[CV] .................. C=10000000000.0, score=0.903579, total= 6.3min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 62.9min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 100.07206721,  474.59572864,  916.21848428,  817.83034785,
         617.01258881,  550.77818711,  750.95050478,  987.65891111,
        1042.1723371 ,  876.86237974,  686.7201468 ,  499.98860929,
         376.55842566]),
 'mean_score_time': array([0.48061652, 0.53899839, 0.14886298, 0.14308383, 0.07476132,
        0.20955586, 0.140995  , 0.22848825, 0.22000208, 0.07280405,
        0.06859157, 0.049366  , 0.03636861]),
 'mean_test_score': array([0.82389503, 0.88535912, 0.91120758, 0.91101026, 0.90943173,
        0.9085438 , 0.90943173, 0.90795185, 0.90844515, 0.90893844,
        0.90933307, 0.90933307, 0.91031965]),
 'mean_train_score': array([0.83824468, 0.92471287, 0.98137524, 0.99718256, 0.99779659,
        0.9981255 , 0.99745608, 0.99744534, 0.99736816, 0.99731344,
        0.99826801, 0.99742382, 0.99757751]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.82390, std: 0.00951, params: {'C': 0.01},
 mean: 0.88536, std: 0.01100, params: {'C': 0.1},
 mean: 0.91121, std: 0.01043, params: {'C': 1.0},
 mean: 0.91101, std: 0.00970, params: {'C': 10.0},
 mean: 0.90943, std: 0.01049, params: {'C': 100.0},
 mean: 0.90854, std: 0.00979, params: {'C': 1000.0},
 mean: 0.90943, std: 0.00938, params: {'C': 10000.0},
 mean: 0.90795, std: 0.00979, params: {'C': 100000.0},
 mean: 0.90845, std: 0.00854, params: {'C': 1000000.0},
 mean: 0.90894, std: 0.00967, params: {'C': 10000000.0},
 mean: 0.90933, std: 0.00880, params: {'C': 100000000.0},
 mean: 0.90933, std: 0.00925, params: {'C': 1000000000.0},
 mean: 0.91032, std: 0.00966, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 1.0} - score: 0.9112
